Tässä Jupyter notebookissa esitellään Latent Dirichlet Allocation-menetelmään (LDA) pohjautuva visualisointi Ylen vuoden 2015 eduskuntavaalien vaalikoneen avoimien kysymysten vastauksista. LDA on tilastollinen menetelmä jolla etsitään aiheita tai n.s. "topicceja" tekstiaineistosta. Menetelmä on "unsupervised", eli se ei vaadi havainnoille luokkia tai vastemuuttujia. Tarkemmin metodista voi lukea esim. [Wikipediasta](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

Visualisointia varten en joutunut juurikaan kirjoittamaan omaa koodia, vaan pystyin käyttämään R- ja Python-kielille tehtyjä valmiita kirjastoja. Ennen kaikkea pohjana toimivat Carson Sievertin LDAvis [R-kirjasto](https://github.com/cpsievert/LDAvis) ja sen havainnollistavat [esimerkit](http://cpsievert.github.io/LDAvis/reviews/reviews.html) sekä Ben Mabeyn [Python-porttaus](https://github.com/bmabey/pyLDAvis) samaisesta kirjastosta ja sen [esimerkit](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb).

Itse LDA-malli on luotu tämän Github-repositoryn juuresta löytyvällä `gen_model.R`-skriptillä. Tuo skripti olettaa että data-kansiosta löytyy korjattu vaalikonedata (katso `README.md`:sta tarkemmat ohjeet) tiedostonimellä `vaalikone_data_fix.csv`, ja luo tämän datan pohjalta data-kansioon visualisoinnin käyttämät json-tiedostot.

Kun tarvittavat tiedostot on luotu, visualisointi luodaan ja näytetään seuraavalla koodinpätkällä:

In [1]:
import json
import numpy as np
import pyLDAvis


def load_R_model(filename):
    with open(filename, 'r') as j:
        data_input = json.load(j)
    data = {'topic_term_dists': data_input['phi'], 
            'doc_topic_dists': data_input['theta'],
            'doc_lengths': data_input['doc.length'],
            'vocab': data_input['vocab'],
            'term_frequency': data_input['term.frequency']}
    return data

movies_model_data = load_R_model('data/vaalikone_lda_input.json')
movies_vis_data = pyLDAvis.prepare(**movies_model_data)
pyLDAvis.display(movies_vis_data)

Kaikille LDA-menetelmän löytämälle 30:lle aiheelle ei ole helppoa keksiä nimeä tai yhdistävää tekijää, mutta esim aiheesta nro 28 löytyvät ruotsinkieliset vastaukset. Topicista 7 löytyy vihertäviä teemoja, kuten ilmastonmuutos, uusiutuvaa energia ja perustulo. Aihe 24 taas liittyy selvästi sukupuolineutraaliin avioliittoon ja sitä ympäröivään keskusteluun ja topic nro 8 Natoon. Menetelmä löytää myös tiettyjä alueellisia teemoja. Esimerkiksi topic nro 25 näyttää käsittelevän Pohjois-Suomen asioita, kun taas topicista 26 löytyvät Turku ja Varsinais-Suomi.

Ymmärtääkseni visualisoinnissa on tehty PCA-projektio kahteen dimensioon ja sitten sijoitettu eri topicit siten että lähellä toisiaan olevat aiheet ovat jokseenkin samankaltaisia. Joissain tapauksissa tämä näyttää toimivan. Esimerkiksi aiheet 6 ja 13 ovat vierekkäin ja sisältävät samanlaisia termejä (esim. tulisi/pitäisi). Sen sijaan jokseenkin keskeltä aihekarttaa löytyy ryteikkö jossa ovat aiheet 7, 11 ja 12 melkein päällekkäin. Topicista 7 löytyy vihreitä vaaliteemoja (esim. perustulo) kun taas topicista 12 löytyy sääntelyn purkamiseen ja työn kannattavuuteen liittyviä talousliberaaleja avainsanoja. Näiden kahden topicin sijoittamista vierekkäin voi pitää hieman outona.